In [1]:
import json

import re
import scipy
import numpy as np

import scipy.sparse

from tqdm import tqdm_notebook as tqdm

In [2]:
enum_role = "环节"
max_seq_len = 300

In [3]:
def role_data_process(dataset):
    """data_process"""
    
    def label_data(data, start, l, _type):
        """label_data"""
        for i in range(start, start + l):
            suffix = "B-" if i == start else "I-"
            if isinstance(data[i], str):
                data[i] = []
            solt = "{}{}".format(suffix, _type)
            if solt not in data[i]:
                data[i].append(solt)
        return data
    
    def replace_control_chars(str):
        if str == '\u200b' or str == '\ufeff' or str == '\ue601' or str == '\u3000':
            return '[UNK]'
        else:
            return str

    output = []
    for d_json in dataset:
        _id = d_json["id"]
        text_a = [
            "，" if t == " " or t == "\n" or t == "\t" else replace_control_chars(t)
            for t in list(d_json["text"].lower())
        ]
        if len(d_json.get("event_list", [])) == 0:
            continue
         ### combine same event type
        event_type_mapping = {}
        for event in d_json.get("event_list", []):
            event_type = event['event_type']
            trigger = event['trigger']
            type_tuple = (event_type, trigger)
            if type_tuple not in event_type_mapping:
                event_type_mapping[type_tuple] = []
            for argument in event["arguments"]:
                if argument not in event_type_mapping[type_tuple]:
                    event_type_mapping[type_tuple].append(argument)

        for type_tuple, arguments in event_type_mapping.items():
            event_type = type_tuple[0]
            trigger = type_tuple[1]
            trigger_text = event_type+f"({trigger})："
            labels = ["O"] * len(text_a)
            for arg in arguments:
                role_type = arg["role"]
                if role_type == enum_role:
                    continue
                argument = arg["argument"]
                start = arg["argument_start_index"]
                labels = label_data(labels, start, len(argument), role_type)
            text_a_trigger = [
                "，" if t == " " or t == "\n" or t == "\t" else t
                for t in list(trigger_text.lower())
            ]
            trigger_label = ["O"] * len(text_a_trigger)
            output.append({
                "id": d_json["id"],
                "sent_id": d_json["sent_id"],
                "text": trigger_text + d_json["text"],
                "tokens": text_a_trigger + text_a,
                "labels": trigger_label+labels
            })
    return output

In [4]:
with open('./resources/duee_fin_train_preprocess.json', 'r', encoding='utf-8') as f:
    dataset = json.loads(f.read())
    preprocess_dataset = role_data_process(dataset)
    count = 0
    duplicate_ids = []
    for i, d_json in enumerate(preprocess_dataset):
        if any([len(label) > 1 for label in d_json['labels'] if isinstance(label, list)]):
            print(i, d_json)
            if d_json["id"] not in duplicate_ids:
                count += 1
                duplicate_ids.append(d_json["id"])

68 {'id': 'e133a2b8ec7b1b95003cc2ca5e8438f6', 'sent_id': 'faa18709a964baf7bfde93d5c7367e8c', 'text': '股份回购(回购)：东瑞制药9月27日耗资14.71万港元回购10.4万股客户端 东瑞制药(1.4,-0.03,-2.10%)（02348.HK）公布，2019年9月27日，公司按每股1.39-1.46港元，耗资14.71万港元回购10.4万股。暴跌62%，公司紧急回应，个人投资者或爆仓，都是券商孖展新规惹的祸？新浪港股讯 10月10日消息，福晟国际暴跌62%，报价0.095港元，单日市值缩水超一半。福晟国际10月8日报价0.38港元，2个交易日内股价已暴跌75%。福晟国际表示，其并不知悉股价或成交量变动之原因，同时董事会确认公司运作一切正常。那么... 2019-10-10 17:14:09 发生了什么？人民币暴涨700点，A股港股拉升，创业板暴涨2.7%，这些股票遭爆炒！全球金融市场振幅加剧，美元、美股大幅跳水之后，人民币资产却上演出独立的“大反转”行情。今日早盘，受到美股市场巨幅震动影响，离岸人民币兑美元刚开始出现了短线下跌的态势。', 'tokens': ['股', '份', '回', '购', '(', '回', '购', ')', '：', '东', '瑞', '制', '药', '9', '月', '2', '7', '日', '耗', '资', '1', '4', '.', '7', '1', '万', '港', '元', '回', '购', '1', '0', '.', '4', '万', '股', '客', '户', '端', '，', '东', '瑞', '制', '药', '(', '1', '.', '4', ',', '-', '0', '.', '0', '3', ',', '-', '2', '.', '1', '0', '%', ')', '（', '0', '2', '3', '4', '8', '.', 'h', 'k', '）', '公', '布', '，', '2', '0', '1', '9', '年', '9', '月', '2', '7', '日', '，', '公', '司', '按', '每', '股', '1', 

3663 {'id': '85cb311c3ab9ad0d85d412d613e97b96', 'sent_id': 'b112fdbbee09d160a4f259d4137a3d28', 'text': '质押(质押)：民生银行(01988-HK)获东方股份通知解除7740万A股质押原标题：民生银行(6.420,0.07,1.10%)(01988-HK)获东方股份通知解除7740万A股质押\n 来源：财华网 [财华社讯]民生银行（01988-HK）公布，近日，中国民生银行股份接到东方集团(3.790,0.07,1.88%)股份有限公司通知，获悉东方股份于2019年6月25日对所持该公司A股普通股7740万股办理了解除质押登记手续。上述解除质押手续已于2019年6月26日在中国证券登记结算有限责任公司北京分公司办理完毕。截至本公告日，东方股份持有该公司普通股约12.8亿股，累计质押该公司股份约11.844亿股，占该公司总股本的2.71%。东方股份的控股股东东方集团有限公司持有该公司普通股3500万股。东方股份与华夏人寿保险股份有限公司于2016年6月29日签署《一致行动协议》，于2018年12月27日签署《一致行动协议之补充协议》，东方有限与华夏人寿于2018年12月27日签署《一致行动协议》。', 'tokens': ['质', '押', '(', '质', '押', ')', '：', '民', '生', '银', '行', '(', '0', '1', '9', '8', '8', '-', 'h', 'k', ')', '获', '东', '方', '股', '份', '通', '知', '解', '除', '7', '7', '4', '0', '万', 'a', '股', '质', '押', '原', '标', '题', '：', '民', '生', '银', '行', '(', '6', '.', '4', '2', '0', ',', '0', '.', '0', '7', ',', '1', '.', '1', '0', '%', ')', '(', '0', '1', '9', '8', '8', '-', 'h', 'k', ')', '获', '东', '方', '股', '份', '通', '知', '解', '除', '7', '7', 

6646 {'id': '04147d14457df1e95c8075f7af4f9a71', 'sent_id': '7d93973fa8a9a4dc00178b8a8b2c3361', 'text': '质押(质押)：珠海赛隆药业股份有限公司关于公司控股股东部分股份质押的公告原标题：珠海赛隆药业(11.510,0.06,0.52%)股份有限公司关于公司控股股东部分股份质押的公告 证券代码：002898\n\n\n 证券简称：赛隆药业\n\n 公告编号：2020-087 珠海赛隆药业股份有限公司 关于公司控股股东部分股份质押的 公告 本公司及董事会全体成员保证信息披露的内容真实、准确、完整，没有虚假记载、误导性陈述或重大遗漏。珠海赛隆药业股份有限公司（以下简称“公司”）于近日接到公司控股股东蔡南桂先生的通知，获悉蔡南桂先生将其持有的本公司部分股份办理了质押手续，具体事项如下： 一、股东股份质押的基本情况 1.股东本次股份质押的基本情况\n2．股东股份累计质押情况 截至本公告披露日，蔡南桂先生及其一致行动人所持质押股份情况如下：\n注：蔡南桂先生与唐霖女士系夫妻关系，为一致行动人，是公司的实际控制人，唐霖女士没有质押公司股票。', 'tokens': ['质', '押', '(', '质', '押', ')', '：', '珠', '海', '赛', '隆', '药', '业', '股', '份', '有', '限', '公', '司', '关', '于', '公', '司', '控', '股', '股', '东', '部', '分', '股', '份', '质', '押', '的', '公', '告', '原', '标', '题', '：', '珠', '海', '赛', '隆', '药', '业', '(', '1', '1', '.', '5', '1', '0', ',', '0', '.', '0', '6', ',', '0', '.', '5', '2', '%', ')', '股', '份', '有', '限', '公', '司', '关', '于', '公', '司', '控', '股', '股', '东', '部', '分', '股', '份', '质', '押', '的', '公', '告', '，', '证', '券', '代', '码', 

9832 {'id': '3f8a2fdad68a26fb0a06ef8efcd6bee5', 'sent_id': '8ba843a9ed5d19f6cfebecde81c258ea', 'text': '高管变动(辞职)：蒂森克虏伯监事会建议由Martina Merz出任临时CEO安装新浪财经客户端第一时间接收最全面的市场资讯→【下载地址】 原标题：蒂森克虏伯监事会建议由监事会主席Martina Merz出任临时CEO 来源：界面新闻 德国时间9月24日深夜，蒂森克虏伯以股市公告的形式对外宣布，公司监事会人事委员会建议CEO基多·克尔克霍夫（Guido Kerkhoff）尽快辞职。公司监事会人事委员会还建议由监事会主席Martina Merz出任临时CEO一职，Siegfried Russwurm则将代理监事会主席一职。此外，目前分管公司材料交易业务的Klaus Keysberg也将于十月起进入董事会并兼管钢铁业务。关于人事委员会的这一套组合拳，监事会将尽快召开紧急会议并做出最终决定。蒂森克虏伯短暂的基多时代即将画上句号。', 'tokens': ['高', '管', '变', '动', '(', '辞', '职', ')', '：', '蒂', '森', '克', '虏', '伯', '监', '事', '会', '建', '议', '由', 'm', 'a', 'r', 't', 'i', 'n', 'a', '，', 'm', 'e', 'r', 'z', '出', '任', '临', '时', 'c', 'e', 'o', '安', '装', '新', '浪', '财', '经', '客', '户', '端', '第', '一', '时', '间', '接', '收', '最', '全', '面', '的', '市', '场', '资', '讯', '→', '【', '下', '载', '地', '址', '】', '，', '原', '标', '题', '：', '蒂', '森', '克', '虏', '伯', '监', '事', '会', '建', '议', '由', '监', '事', '会', '主', '席', 'm', 'a', 'r', 't', 'i', 'n', 'a', '，', 'm', 'e', 'r', 'z', '出', '

In [5]:
print(count)
print(len(dataset))

311
12076


In [6]:
with open('./resources/duee_fin_dev_preprocess.json', 'r', encoding='utf-8') as f:
    dataset = json.loads(f.read())
    preprocess_dev_dataset = role_data_process(dataset)
    count = 0
    duplicate_ids = []
    for i, d_json in enumerate(preprocess_dev_dataset):
        if any([len(label) > 1 for label in d_json['labels'] if isinstance(label, list)]):
            print(i, d_json)
            if d_json["id"] not in duplicate_ids:
                count += 1
                duplicate_ids.append(d_json["id"])

22 {'id': '6840d385c03e24f37064faa6c6eae757', 'sent_id': '162ade32ecf0466de80d1c09939f8b3e', 'text': '质押(质押)：方盛制药：控股股东张庆华解除质押约1779.65万股，再质押1990万股每经AI快讯，方盛制药（SH 603998，收盘价：6.87元）7月7日晚间发布公告称，公司控股股东暨实际控制人张庆华先生于7月7日将约1779.65万股解除质押。同时，张庆华与长沙银行股份有限公司高信支行办理完成了股份质押手续，将其持 有的1990万股进行了质押。截至本公告披露日，张庆华先生直接持有公司的股份总数约为1.56亿股，占公司股份总数的35.96%；张庆华先生质押的公司股份总数约为1.4亿股，占其直接持有公司股份总数的89.77%，占公司股份总数的32.28%。2019年年报显示，方盛制药的主营业务为医药制造、医药商业，占营收比例分别为：93.78%、4.22%。方盛制药的董事长是张庆华，男，46岁，毕业于湖南中医药大学，大专学历。方盛制药的总经理是肖汉卿，男，46岁，毕业于湖南工程学院，大专学历。(记者 李星)', 'tokens': ['质', '押', '(', '质', '押', ')', '：', '方', '盛', '制', '药', '：', '控', '股', '股', '东', '张', '庆', '华', '解', '除', '质', '押', '约', '1', '7', '7', '9', '.', '6', '5', '万', '股', '，', '再', '质', '押', '1', '9', '9', '0', '万', '股', '每', '经', 'a', 'i', '快', '讯', '，', '方', '盛', '制', '药', '（', 's', 'h', '，', '6', '0', '3', '9', '9', '8', '，', '收', '盘', '价', '：', '6', '.', '8', '7', '元', '）', '7', '月', '7', '日', '晚', '间', '发', '布', '公', '告', '称', '，', '公', '司', '控', '股', '股', '东', '暨', '实',

In [7]:
print(count)
print(len(dataset))

63
1946


In [8]:
def load_dict(dict_path):
    """load_dict"""
    vocab = {}
    for line in open(dict_path, 'r', encoding='utf-8'):
        value, key = line.strip('\n').split('\t')
        vocab[key] = int(value)
    return vocab

In [9]:
label_vocab = load_dict(dict_path='./dictionary/role_tag.dict')

In [10]:
def cal_cover_overlap_conflict(preprocess_dataset):
    coverage_score, overlap_score, conflict_score = 0.0, 0.0, 0.0
    for d_json in tqdm(preprocess_dataset, total=len(preprocess_dataset)):
        labels = d_json['labels']
        label_array = [-1 * np.ones(len(labels))]
        for i, label in enumerate(labels):
            if isinstance(label, str):
                label_array[0][i] = 120
            if isinstance(label, list):
                if len(label_array) < len(label):
                    for j in range(len(label)-len(label_array)):
                        label_array.append(120 * np.ones(len(labels)))
                for j in range(len(label)):
                    l = re.sub(r"\(.*\)", "", label[j])
                    label_array[j][i] = label_vocab.get(l, 120)
        L = np.array(label_array).T
        L_sparse = scipy.sparse.csr_matrix(L + 1)
        covered_data_points = np.ravel(np.where(L_sparse.sum(axis=1) != 0, 1, 0))
        coverage_score += covered_data_points.sum() / L_sparse.shape[0]
        overlapped_data_points = np.where(np.ravel((L_sparse != 0).sum(axis=1)) > 1, 1, 0)
        overlap_score += overlapped_data_points.sum() / L_sparse.shape[0]
        m = scipy.sparse.diags(np.ravel(L_sparse.max(axis=1).todense()))
        conflicted_data_points = np.ravel(np.max(m @ (L_sparse != 0) != L_sparse, axis=1).astype(int).todense())
        conflict_score += conflicted_data_points.sum() / L_sparse.shape[0]
        
    print(f"coverage_score: {coverage_score / len(preprocess_dataset)}, overlap_score: {overlap_score / len(preprocess_dataset)}, conflict_score: {conflict_score / len(preprocess_dataset)}")

In [11]:
cal_cover_overlap_conflict(preprocess_dataset)


coverage_score: 1.0, overlap_score: 0.3305387475395495, conflict_score: 0.0


In [12]:
cal_cover_overlap_conflict(preprocess_dev_dataset)


coverage_score: 1.0, overlap_score: 0.3155699721964782, conflict_score: 0.0
